In [1]:
from csv_psfs import *
import svd_model as svm

import forward_model as fm

import numpy as np
import scipy.io

# My SVD
Grab the PSF stack, do the SVD, test on the hydra. Simple as.

In [2]:
psf_directory = "/home/dshteinbok/nV3_PSFs"
# TODO: where to save

img_dims = (486, 648)

In [3]:
# load the PSFs
psfs = pad_as_center(load_from_dir(psf_directory), img_dims[0], img_dims[1])

In [4]:
ref_image_index=4
psfs_reg,shifts=svm.register_psfs(psfs, psfs[:,:,ref_image_index],dct_on=True)

Removing background

done

registering

done registering



In [5]:
rank=12
comps, weights_interp=svm.calc_svd(psfs_reg,shifts,rank)

creating matrix

done

starting svd...

check values of ymat
interpolating...

done




In [6]:
# Normalize weights to have maximum sum through rank of 1
weights_norm = np.sum(weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:],0).max()
weights = weights_interp/weights_norm;

#normalize by norm of all stack. Can also try normalizing by max of all stack or by norm of each slice
h=comps/np.linalg.norm(np.ravel(comps))

In [9]:
h.shape

(486, 648, 12)

In [10]:
weights.shape

(486, 648, 12)

In [7]:
h_dict = {"array_out": h}
scipy.io.savemat("../data/nV3_h.mat", h_dict)

In [11]:
weights_dict = {"array_out": weights}
scipy.io.savemat("../data/nV3_weights.mat", weights_dict)